In [1]:

import json
import re
import gzip
import pickle
from baseline.data_process import Example

In [2]:
# 读取2019年的原训练数据集
with open('./data2019/big_train_data.json', 'r', encoding='utf-8') as reader:
    full_data_2019 = json.load(reader) 


In [7]:
def convert_context_to_2020(context_2019):
    '''
    把2019年案例内容根据标点符号分句，构造成2020年的原始标准格式
    '''
    context_2020=[]
    pattern = r',|\.|\:|;|!|\?|:|，|。|：|；|！|？'
    result_list = re.split(pattern, context_2019)
    context_2020.append(result_list[0])
    context_2020.append(result_list)
    return [context_2020]

def get_supporting_facts_sen_id(context_2019, answer_start, answer_text):
    
    '''
    把2019年案例的回答依据，构造成2020年的格式。
    '''
    pattern = r',|\.|\:|;|!|\?|:|，|。|：|；|！|？'
    sen_list = re.split(pattern, context_2019)
    answer_index = 0
    sen_id = {}
    for i, sen in enumerate(sen_list):
        # 在分句后的案例内容中查找行号
        index = sen.find(answer_text)
        if index >=0 :
            sen_id[i] = abs(answer_start - answer_index - index)
        answer_index += len(sen)
    supporting_fact = []
    supporting_fact.append(sen_list[0])
    if len(sen_id) ==0 :
        supporting_fact.append(-1)
    else:
        supporting_fact.append( min(sen_id, key=sen_id.get))
    return [supporting_fact]

train_2019 = []
for i, case_2019 in enumerate(full_data_2019['data']):
    case_2020= {}
    # 遍历qas, 
    question_2019 = {}
    # 先取question_2019为true
    for qa in case_2019['paragraphs'][0]['qas']:
        # 过滤answers为空的question
        if qa['is_impossible']=='true' and  len(qa['answers']) > 0 and qa['answers'][0]['answer_start'] !=-1:
            question_2019=qa
            break
    if len(question_2019)==0:       
        for qa in case_2019['paragraphs'][0]['qas']:
            # 过滤answers为空的question
            if qa['is_impossible']=='false' and len(qa['answers']) > 0 and qa['answers'][0]['answer_start'] !=-1:
                    question_2019=qa
                    answer_txt =qa['answers'][0]['text']
                    # 取出answers的text为yes或no的第一个question
                    if answer_txt.lower() == 'yes' or answer_txt.lower() == 'no':
                        break
    case_2020['_id'] = i+5055
    case_2020['context'] = convert_context_to_2020(case_2019['paragraphs'][0]['context'])
    case_2020['question'] = question_2019['question']
    case_2020['answer'] = question_2019['answers'][0]['text']
    case_2020['supporting_facts'] =get_supporting_facts_sen_id(case_2019['paragraphs'][0]['context'],
                                                           question_2019['answers'][0]['answer_start'],
                                                           question_2019['answers'][0]['text'])
    
    train_2019.append(case_2020)


In [8]:
# 把处理好的2019年数据集，保存到data文件夹下
with open('./data/train_2019.json', 'w', encoding='utf-8') as writer:
    data=json.dumps(train_2019,ensure_ascii=False)
    writer.write(data)

In [6]:
# 把处理好的2019年数据集，合并2020年数据集，输出到新文件中
# with open('./data/train_2019_to_2020.json', 'w', encoding='utf-8') as writer:
#     with open('./data/train.json', 'r', encoding='utf-8') as reader:
#         train_2020=json.load(reader)
#         train_2020.extend(train_2019)
#         data=json.dumps(train_2020,ensure_ascii=False)
#         writer.write(data)